In [1]:
# if lights are off when a color transition starts, or for a brightness transition to zero, transition will fail.
# So we need a way to initiate a transition based on expected state.

# But reworking the script to do that is... kinda big. This is the quick and dirty version.
# Assumes the transition is 1hr+ long. 

In [2]:

%load_ext autoreload
%autoreload 2

import datetime
import random
import time


from tradfri import Tradfri

#%aimport tradfri

Todo:
look for all 'todo' instances here.


... this is not turning out to be as 'quick and dirty' as I thought.


figure out multithreading. https://stackoverflow.com/questions/2846653/how-to-use-threading-in-python

#### Approach:

Process:
Plan transition for [brightness, color] for each light based on start & end times
Skip through transition until we reach current time. Do short transition (30s?) to that. 


In [3]:
### DEBUG
from pprint import pprint
DEBUG = 1

In [4]:
### CONFIG:

evening_start_time = datetime.time(hour=17, minute=30)

# 'quick' initial transition to bring lights in line with overall plan. think a minute-ish.
# definitely don't go more than 15 min or you're not 'catching up' fast enough.
initial_transition_duration = datetime.timedelta(seconds=6)

# different times for end of color transition and end of brightness transition:
evening_color_end_value = 2200
evening_color_end_time = datetime.time(hour=23)

evening_brightness_end_value = 0
evening_brightness_end_time = datetime.time(hour=23, minute=45)

# NOTE: don't transition over midnight, or this will break. Like I said, quick and dirty.
#(Modify to use datetimes if you need to transition spanning midnight.
#  Or check if start is after end & +1 the date.)


# we'll extend transitions by a random amount, between 0 and this many seconds,
#  to prevent everything from switching off at once.
# recommend 15 minutes = 900 seconds
random_delay_range = 900



In [5]:
# basically just a record of daytime values for the office. also defined in homeassistant as an automation.
# *not* the same names as in light-schedule.py; these are more like the entity ids.
daytime_values = {"office_table": {
                            "brightness": 230,
                            "color": 4000
                            },
           "desk_lamp": {
                           "brightness": 200,
                           "color": 3200
                       },
           "geo_desk": {
                           "brightness": 130,
                           "color": 3350
                       },
           "floor_uplight": {
                           "brightness": 230,
                           "color": 3800
                       },
           "monitor_left": {
                           "brightness": 130,
                           "color": 3700
                       },
           "monitor_right": {
                           "brightness": 130,
                           "color": 3700
                       }
          }


In [6]:
# Do some setup / initial math and stuff.

today = datetime.datetime.now().date()

# make datetimes from dates. Use these internally instead of vals above.
e_start = datetime.datetime.combine(today, evening_start_time)
c_end = datetime.datetime.combine(today, evening_color_end_time)
b_end = datetime.datetime.combine(today, evening_brightness_end_time)

# timedeltas
color_duration = c_end - e_start
brightness_duration = b_end - e_start

# put these into a more easily programmatically-navigable structure.
details = {
              "color": {
                        "value": evening_color_end_value,
                        "end_time": evening_color_end_time,
                        "duration": color_duration
                       },
              "brightness": {
                        "value":evening_brightness_end_value,
                        "end_time": evening_brightness_end_time,
                        "duration": brightness_duration
              }
}


In [7]:
# list of each transition we'll have to do. these are *start* values.
transitions = []
for i in daytime_values:
    #print(i)
    transitions.append({"name": i, "type": "brightness", "value": daytime_values[i]['brightness']})
    transitions.append({"name": i, "type": "color", "value": daytime_values[i]['color']})

In [8]:
transitions[0]

{'name': 'office_table', 'type': 'brightness', 'value': 230}

In [41]:
### multithreading demo here

In [42]:
def get_val(transition):
    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    if transition['type'] == 'brightness':
        val = device.get_brightness()
    else: # it's color
        val = device.get_temp_kelvin()

    #print("gv()", transition['name'], transition['type'], val, sep='\t')
    return (transition['name'], transition['type'], val)

In [43]:
# non-multithreaded version
for i in transitions[0:3]:
    val = get_val(i)
    #print(i['name'], i['type'], val[1])

In [19]:
from multiprocessing.dummy import Pool as ThreadPool
# maybe problematic if we have hundreds of transitions, but we don't
pool = ThreadPool(len(transitions))

In [45]:
# multithreaded version. this is actually pretty easy.
results = pool.map(get_val, transitions)

In [46]:
results

[('office_table', 'brightness', 230),
 ('office_table', 'color', 4000),
 ('desk_lamp', 'brightness', 200),
 ('desk_lamp', 'color', 3205),
 ('geo_desk', 'brightness', 130),
 ('geo_desk', 'color', 3356),
 ('floor_uplight', 'brightness', 230),
 ('floor_uplight', 'color', 3802),
 ('monitor_left', 'brightness', 130),
 ('monitor_left', 'color', 3704),
 ('monitor_right', 'brightness', 130),
 ('monitor_right', 'color', 3704)]

In [47]:
## end multithreading demo

In [16]:
# ok, here we go.

def catch_up(transition, catch_up_time=None):
    # 'resumes' an existing transition.
    # if catch_up_time (datetime) is specified, catches up to that time, 
    #   else catches up to current time. 
    
    # catch_up_time is only really useful for debugging / development, 
    #   since when the regular plan is resumed, current time wil be used.
    # probably best to remove it later.
    
    
    ### todo: function too big, split up?
    ### todo: check if current time is between start and end times before going through the whole plan.
    
    ### todo: make initial transition shorter if # of steps is low. min 1 second per step. 
    
    # BUG: won't work if lights are off, even though transitions from 0 to >0 are supported from off
    
    # BUG: 
    # error when transitioning a 'color' value.
    """
    ~\Documents\homeassistant\tradfri-lights\tradfri.py in calculate_steps_by_granularity(current_attrs, new_attr)
    281 
    282                 # skip if current state and transition state are equal:
--> 283                 if new_attr[key] == current_attrs[key]:

    KeyError: 'color'
    """
    
    # BUG:
    """
    ~\Documents\homeassistant\tradfri-lights\tradfri.py in execute_transition(self, plan)
    351         if self.debug > 0:
    352             print("time until start", time_until_start)
--> 353         time.sleep(time_until_start.total_seconds() - 0.1)
    354 
    355         step_sleep = MIN_STEP_DURATION.total_seconds()

    ValueError: sleep length must be non-negative
    """
    
    # generally only happens in dev, but good to catch. 
    now = datetime.datetime.now().date()
    if now != today:
        print("WHOOPS: 'today' value is not actually today, so transitions will fail")
        print("make sure to run the cell for 'do some setup'.")
        # other possibility: transition occurs over midnight, or is > 1 day long.
        return None
    
    
    # step 1 in randomizing
    if DEBUG:
        time.sleep(random.uniform(0,1))
    else:
        time.sleep(random.uniform(0,9))

    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    
    #print(details[i['type']])
    transition_type = transition['type']
    start_attr = {transition_type: transition['value']}
    final_attr = {transition_type: details[transition_type]['value']}
    duration = details[transition_type]['duration']
    
    if transition_type == "brightness":
        cur_value = {"brightness": device.get_brightness()}
    else:
        cur_value = {"color": device.get_color()}
    if DEBUG:
        print("cur value:", cur_value)
            
    
    # randomize a bit more.
    random_delay = random.uniform(0, random_delay_range)
    duration += datetime.timedelta(seconds = random_delay)
    
    plan = device.plan_transition(final_attr, duration, start_time=e_start, start_attr=start_attr)
    if DEBUG:
        print("\trandomized delay:", random_delay)
        print("\tnew end time:", e_start + duration)
        print("total plan steps:", len(plan['plan']))
    
    # debug
    #pprint(plan['plan'][-3:])
    
    # find the point in the the transition *after* our short initial transition to current value
    if catch_up_time:
        benchmark_time = catch_up_time + initial_transition_duration
    else:
        benchmark_time = datetime.datetime.now() + initial_transition_duration
    
    #debug
    min_plan_time = plan['plan'][0]['step_start_time']
    max_plan_time = plan['plan'][-1]['step_start_time']
    
    #print("benchmark time:", benchmark_time)
    #print("min time in plan:", min_plan_time)
    #print("max time in plan:", max_plan_time)
    
    if not (benchmark_time > min_plan_time and max_plan_time > benchmark_time):
        print("Error: catch-up time not between plan start and end times.")
        print("catch-up time:", benchmark_time)
        print("min time in plan:", min_plan_time)
        print("max time in plan:", max_plan_time)
        return None
    
    current_step_no = None
    # find out how far into the transition we are.
    for i in plan['plan']: 
        if i['step_start_time'] >= benchmark_time:
            current_step_no = i['step_number'] - 1
            break
    
    if current_step_no is None:
        # probably beyond the end of the transition.
        print("no step found!")
        
        ### write me
        
    elif current_step_no == 0:
        # may or may not have started yet? do we need to special case this?
        print("cur step is zero!")
        
        ### write me
        
    elif current_step_no < 0:
        # definitely haven't started yet
        print("cur step is negative! step:", current_step_no)
        
        ### write me
    
    else: #continue
        
        if DEBUG:
            total_steps = len(plan['plan'])
            pct_complete = round(100*(current_step_no/total_steps))
            print("current transition is", pct_complete, "percent completed")
    
        current_step_vals = plan['plan'][current_step_no]

        print("cur step:", current_step_vals)

        step_value = {transition_type: plan['plan'][current_step_no][transition_type]}


        if cur_value != step_value:
            # Do a quick transition to these values
            if DEBUG: print("catching up to the following values over the next", 
                initial_transition_duration.seconds, 
                "seconds:", step_value)
            device.transition(step_value, initial_transition_duration)
            if DEBUG: print("caught up.", end=" ")

        if DEBUG: print("resuming normal schedule")
        
        if DEBUG:
            print("csn #:", current_step_no, "of", total_steps)
        
        # Then, resume normal transition, starting from current step / values.
        new_plan = plan['plan'][current_step_no:]
        plan['plan'] = new_plan
        device.execute_transition(plan)
    
    if DEBUG:
        print("")

In [ ]:
pool = ThreadPool(len(transitions))
results = pool.map(catch_up, transitions)

cur value: {'color': 3704}
cur value: {'brightness': 200}
	randomized delay: 694.2340671218867
	new end time: 2018-09-17 23:56:34.234067
total plan steps: 200
current transition is 76 percent completed
cur step: {'brightness': 46.0, 'step_start_time': datetime.datetime(2018, 9, 17, 22, 25, 43, 589010), 'step_number': 153}
catching up to the following values over the next 6 seconds: {'brightness': 46.0}
cur value: {'color': 3205}
cur value: {'color': 3356}
cur value: {'brightness': 130}
	randomized delay: 328.6277356822909
	new end time: 2018-09-17 23:50:28.627736
total plan steps: 130
current transition is 78 percent completed
cur step: {'brightness': 28.0, 'step_start_time': datetime.datetime(2018, 9, 17, 22, 25, 36, 87729), 'step_number': 101}
catching up to the following values over the next 6 seconds: {'brightness': 28.0}
cur value: {'color': 3704}
cur value: {'color': 4000}
cur value: {'brightness': 130}
	randomized delay: 450.0534689833803
	new end time: 2018-09-17 23:52:30.05346

In [17]:
for n, i in enumerate(transitions[1:2]):
    print("this cell i:", i)
    catch_up(i) #, datetime.datetime(2018, 9, 11, 23, 1))

this cell i: {'name': 'office_table', 'type': 'color', 'value': 4000}
cur value: {'color': 4000}


KeyError: 'color'

In [ ]:
def planner(transition):
    # half ass extracted this from the catch_up(). it's still there too.
    
    entity = 'light.' + transition['name']
    # instantiate
    device = Tradfri(entity)
    
    #print(details[i['type']])
    transition_type = transition['type']
    final_attr = {transition_type: details[transition_type]['value']}
    duration = details[transition_type]['duration']
    
    
    # usage:
    # plan_transition(self, new_attr, duration, start_time=None)
    
    # randomize a bit more.
    duration + datetime.timedelta(seconds = random.uniform(0, random_delay_range))
    plan = device.plan_transition(final_attr, duration, e_start)
    
    return plan

In [ ]:
for i in transitions[0:1]:
    plan = planner(i)
    print("steps:", len(plan['plan']))
pprint(plan['details'])
pprint(plan['plan'][0:5])
print("...")
pprint(plan['plan'][-5:])

In [11]:
import requests
import json
import pandas as pd
API_URL = "http://192.168.132.162:8123/api/"

In [12]:
# this one goes into tradfri.py class

def get_all_light_states(self):
    # gets state for all entities with entity_ids starting with 'light.'
    # Can't differentiate between individual lights and groups of lights, though. I don't think.
    # for my *specific* config, since I'm using mostly white balance bulbs, 
    #  these have supported_features = 35, and groups have 33. (but single-color bulbs also have 33.)
    
    # supported_features is an integer as binary array of flags - 
    # https://community.home-assistant.io/t/supported-features/43696
    
    # get all states
    data = self.apireq('states')
    data = json.loads(data.text)
    
    # filter to 'light.x' entities only
    # note this has a lot of stuff returned. 
    return [i for i in data if i['entity_id'].startswith('light.') and i['attributes']['supported_features'] != 33]
    
    
# this is a convenience function outside of a tradfri class instance.
# compare with get_all_light_states in tradfri. 
def show_light_states():
    API_URL = "http://192.168.132.162:8123/api/"
    
    data = requests.get(API_URL + 'states')
    data = json.loads(data.text)
    if data:
        finaldata = []
        for i in data:
            #print(i)
            if i['entity_id'].startswith('light.') and i['attributes']['supported_features'] != 33:
                if i['state'] == 'on':
                    # should use a regex to trim entity id starting with light.x
                    row = {"entity_id": i['entity_id'].replace('light.',""),
                           "state": i['state'],
                           "color": round(1000000 / i['attributes']['color_temp']), 
                           "brightness": i['attributes']['brightness']}
                else:
                    row = {"entity_id": i['entity_id'],
                           "state": i['state'],
                           "color": None,
                           "brightness": None}
                finaldata.append(row)

        df = pd.DataFrame(finaldata, columns=['entity_id', 'state', 'brightness', 'color'])
        df = df.set_index('entity_id')
        df = df.sort_values(by='entity_id') #by=['entity_id'])
        return df
    
        """
        #print("entity id, state, brightness, color temp:")
        #for i in finaldata:
            # convert to kelvin
            if i['color'] is not None:
                color = round(1000000 / i['color'])
            else:
                color = None
            print(i['entity_id'], i['state'], i['brightness'], color, sep='\t')
        """


In [13]:
show_light_states()

,state,brightness,color
entity_id,,,
desk_lamp,on,200,3205
floor_uplight,on,230,3802
geo_desk,on,130,3356
monitor_left,on,130,3704
monitor_right,on,130,3704
office_table,on,230,4000
